In [1]:
!pip install pandas groq tiktoken getpass


Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement getpass (from versions: none)
ERROR: No matching distribution found for getpass


In [ ]:
import getpass
import time

In [4]:
groq_api_key = getpass.getpass(prompt="Insira a tua GROQ API_KEY: ")


Insira a tua GROQ API_KEY: ········


In [ ]:
import os, json, pandas as pd
from glob import glob
from groq import Groq

# Configurações do modelo
client = Groq(api_key=groq_api_key)
model = "meta-llama/llama-4-scout-17b-16e-instruct"
total_tokens_used = 0

# Funções de avaliação
def run_llama_prompt(prompt: str) -> str:
    global total_tokens_used
    completion = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=512
    )
    usage = getattr(completion, 'usage', None)
    if usage and hasattr(usage, 'total_tokens'):
        total_tokens_used += usage.total_tokens
    return completion.choices[0].message.content.strip()

def validate_score(score: str) -> float:
    try:
        val = float(score)
    except:
        return 0.0
    return max(-5.0, min(val, 10.0))

def score_skills(skills: str, job_desc: str) -> float:
    prompt = f"Given the candidate's skills: {skills}\nAnd the job description: {job_desc}\nRate on -5 to 10. Respond with just the number."
    return validate_score(run_llama_prompt(prompt))

def score_education(education: str, job_desc: str) -> float:
    prompt = f"Given the candidate's education: {education}\nAward extra points for high GPA (no penalties for low).\nJob desc: {job_desc}\nRate on -5 to 10. Respond with just the number."
    return validate_score(run_llama_prompt(prompt))

def score_experience(experience: str, job_desc: str) -> float:
    prompt = f"Given the candidate's experience: {experience}\nAnd the job description: {job_desc}\nRate on -5 to 10. Respond with just the number."
    return validate_score(run_llama_prompt(prompt))

def score_summary(summary: str, job_desc: str) -> float:
    prompt = f"Given the candidate's summary: {summary}\nAnd the job description: {job_desc}\nRate on -5 to 10. Respond with just the number."
    return validate_score(run_llama_prompt(prompt))

# Diretórios
job_folder = 'JobDescriptions_json'
cv_folder = 'CV_json'
output_csv = 'profiles_scored.csv'

# Inicializar CSV (remover anterior se existir)
if os.path.exists(output_csv):
    os.remove(output_csv)
first_write = True

# Carregar job descriptions
job_files = sorted(glob(os.path.join(job_folder, 'JobDescription*.json')))
jobs = []
for path in job_files:
    with open(path, 'r', encoding='utf-8') as f:
        jobs.append(json.load(f))

# Carregar todos os CVs
cv_files = sorted(glob(os.path.join(cv_folder, 'cv_*.json')))
for cv_path in cv_files:
    with open(cv_path, 'r', encoding='utf-8') as f:
        profile = json.load(f)

    for job in jobs:
        job_text = (
            job.get('about_the_role', '') + ' ' +
            job.get('key_responsibilities', '') + ' ' +
            job.get('requirements', '')
        )
        w = job.get('weights', {"summary":1, "experience":1, "education":1, "skills":1})
        total_w = sum(w.values()) if sum(w.values()) != 0 else 1

        s_sk = score_skills(profile.get('skills', ''), job_text)
        s_ed = score_education(profile.get('education', ''), job_text)
        s_ex = score_experience(profile.get('experience', ''), job_text)
        s_sm = score_summary(profile.get('summary', ''), job_text)

        final = (
            w.get('skills',0)*s_sk +
            w.get('education',0)*s_ed +
            w.get('experience',0)*s_ex +
            w.get('summary',0)*s_sm
        ) / total_w

        row_df = pd.DataFrame([{
            'candidate_name': profile.get('name', ''),
            'job_title': job.get('title', ''),
            'final_score': final
        }])
        row_df.to_csv(output_csv, mode='a', header=first_write, index=False)
        first_write = False

# Mostrar total de tokens usados
print(f"\n Avaliação completa! Tokens usados: {total_tokens_used}")
print(f"Resultados salvos em: {output_csv}")


KeyboardInterrupt: 